In [1]:
# import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
#these functions are consulteded from the folowwing link: https://www.kaggle.com/code/neerajmohan/randomforest-model-with-bayesian-optimization
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report,precision_score, recall_score, f1_score, accuracy_score, accuracy_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization

from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

#cross validation to test the mean accuracy of the changed hyperparameters on the k ammount of cross validation folds.
def stratified_kfold_score(clf,X,y,n_fold):
    #strat_kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=1)
    #score = cross_val_score(clf, X, y, cv=strat_kfold, scoring='accuracy', n_jobs=-1)
    #this is not needed scince the standard method is stratefied k fold
    
    score = cross_val_score(clf, X, y, cv=n_fold, scoring='accuracy', n_jobs=-1)
    return score.mean()


#makinging the random forest tree and calling the fucntion for cross valdiation with 10 cross validations
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestClassifier(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    score = stratified_kfold_score(clf,train_X, train_y,10)    
    return score

In [2]:
# Read dataset
clean_df = pd.read_csv('dataset.csv', sep=",")
X = clean_df.drop(columns=['Respiratory Rate (1/min)', 'Ventilation (sL/min)'])
y = clean_df[['Respiratory Rate (1/min)', 'Ventilation (sL/min)']]

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Creating an evaluated model

# Create Bayesian Optimization
rf_bo = BayesianOptimization(bo_params_rf, {
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

#getting best hyperparameters from bayesian optimization
results = rf_bo.maximize(n_iter=250, init_points=20, acq='ei')
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

#making random forest model with the best hyperparameters
rf_bo = RandomForestClassifier(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

# fitting the rf with the best hyperparameters according to the bayesian 
rf_bo.fit(X_train, y_train)

In [ ]:
# predict on test data
y_pred = rf.predict(X_test)

In [ ]:
# calculate metrics
mae = mean_absolute_error(y_test[target], y_pred)
mape = mean_absolute_percentage_error(y_test[target], y_pred)
mse = mean_squared_error(y_test[target], y_pred)
rmse = np.sqrt(mean_squared_error(y_test[target], y_pred))
r2 = r2_score(y_test[target], y_pred)
n = len(y_test[target])
p = X_test.shape[1]
r2_adj = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
# print metrics
print(f'mae: {mae:.2f}')
print(f'mape: {mape:.2f}')
print(f'mse: {mse:.2f}')
print(f'rmse: {rmse:.2f}')
print(f'r2: {r2:.2f}')
print(f'r2_adj: {r2_adj:.2f}')